# Employee ETL Pipeline - Silver Layer

Architecture Overview:  
![silver.jpg](./silver.jpg "silver.jpg")

In this layer we are focusing on transformation of the data - data cleaning, grouping, joins and data enrichment.    
See more on _Medallion Architecture_: https://www.databricks.com/glossary/medallion-architecture

## Import Libraries

In [0]:
from datetime import datetime

from pyspark.sql import functions as F
from pyspark.sql.functions import count, col, sum as spark_sum
from pyspark.sql import Window

## Get Data from Bronze Layer

In [0]:
employees = spark.read.table("hrdata.bronze.employees")

In [0]:
display(employees.limit(5))

## Data Cleaning and Data Quality Checks
Making sure the data does not include unrealistic values or duplicates.

### Duplicates

In [0]:
DUPLICATE_KEY_COLUMNS = [
  "first_name",
  "last_name",
  "email",
  "date_of_birth",
  "phone_number"
]

In [0]:
employees_counts = employees.groupBy(DUPLICATE_KEY_COLUMNS).agg(count("*").alias("count"))
employees_duplicate_keys = employees_counts.filter(col("count") > 1).drop("count")
display(employees_duplicate_keys)

In [0]:
# Join back to the original DataFrame to get ALL duplicate rows
employees_duplicates = employees.join(employees_duplicate_keys, on=DUPLICATE_KEY_COLUMNS, how="inner")

print("--- Found Duplicate Records (All Occurrences) ---")
display(employees_duplicates)
print(f"Total duplicate records found: {employees_duplicates.count()}")

#### Remove duplicates

In [0]:
employees_deduplicated = employees.dropDuplicates(subset=DUPLICATE_KEY_COLUMNS)
print(f"Remaining records: {employees_deduplicated.count()}")

### Data Validation

#### Salary

In [0]:
SALARY_MIN = 30000
SALARY_MAX = 300000

# filtering salaries lower than SALARY_MIN or higher than SALARY_MAX
unrealistic_salaries = employees_deduplicated.filter((col("salary") > SALARY_MAX) | (col("salary") < SALARY_MIN))
display(unrealistic_salaries[
    ["first_name", "last_name", "email", "department", "salary"]
])

In [0]:
filtered_employees = employees_deduplicated.filter((col("salary") <= SALARY_MAX) & (col("salary") >= SALARY_MIN))
print(f"Remaining records: {filtered_employees.count()}")

#### Email

In [0]:
# filtering unrealistic emails

# without [et]
unrealistic_emails = filtered_employees.filter(
    ~filtered_employees.email.contains("@")
)

# emails without point
unrealistic_emails = unrealistic_emails.union(
    filtered_employees.filter(~filtered_employees.email.contains("."))
)

display(unrealistic_emails
    ["first_name", "last_name", "email", "department"]
)

In [0]:
# removing records with invalid email address - filtering out inrealistic_emails
filtered_employees = filtered_employees.join(unrealistic_emails, on="email", how="left_anti")
print(f"Remaining records: {filtered_employees.count()}")

#### Hire Date

In [0]:
HIRE_DATE_MIN = "1990-01-01"
HIRE_DATE_MAX = datetime.now().strftime("%Y-%m-%d")

In [0]:
# filtering hire_date lower than HIRE_DATE_MIN or higher than HIRE_DATE_MAX
unrealistic_hire_dates = filtered_employees.filter((col("hire_date") > HIRE_DATE_MAX) | (col("hire_date") < HIRE_DATE_MIN))
display(unrealistic_hire_dates
    ["first_name", "last_name", "email", "department", "hire_date"]
)

In [0]:
# removing unrealistic hire dates
filtered_employees = filtered_employees.join(unrealistic_hire_dates, on="hire_date", how="left_anti")
print(f"Remaining records: {filtered_employees.count()}")

#### Birth Date

In [0]:
BIRTH_DATE_MIN = "1950-01-01"
BIRTH_DATE_MAX = "2008-01-01"

In [0]:
# filter for unrealistic birth dates
unrealistic_birth_dates = filtered_employees.filter((col("date_of_birth") > BIRTH_DATE_MAX) | (col("date_of_birth") < BIRTH_DATE_MIN))
display(unrealistic_birth_dates
    ["first_name", "last_name", "email", "department", "date_of_birth"]
)

In [0]:
# removing unrealistic birth dates
filtered_employees = filtered_employees.join(unrealistic_birth_dates, on="date_of_birth", how="left_anti")
print(f"Remaining records: {filtered_employees.count()}")

## Add New Columns

### Age

In [0]:
# Add age column
filtered_employees = filtered_employees.withColumn("age", F.round((F.datediff(F.current_date(), col("date_of_birth")) / 365), 0))
display(filtered_employees
    ["first_name", "last_name", "email", "department", "date_of_birth", "age"].limit(3)
)

### Years at the Company

In [0]:
# add new column with the years spent at the company
filtered_employees = filtered_employees.withColumn("years_at_company", F.round((F.datediff(F.current_date(), col("hire_date")) / 365), 1))
display(filtered_employees
    ["first_name", "last_name", "email", "department", "hire_date", "years_at_company"].limit(3)
)

### Validity

In [0]:
# Add valid_from column - in this case this is the hire_date
filtered_employees = filtered_employees.withColumn("valid_from", col("hire_date"))
display(filtered_employees
    ["first_name", "last_name", "email", "department", "hire_date", "valid_from"].limit(3)
)

In [0]:
# add valid_to - in this case this is always empty
filtered_employees = filtered_employees.withColumn("valid_to", F.lit(""))
display(filtered_employees
    ["first_name", "last_name", "email", "department", "hire_date", "valid_from", "valid_to"].limit(3)
)